## <p style="text-align: center;"><span style="color:maroon">MScA 32010 Linear Algebra - Course Project</span></p>
---
# <p style="text-align: center;"> Predicting Migration Flows Into and Out of Cook County, IL </p>
---

The goal of this project is to use concepts of linear algebra to predict migration patterns into and out of Cook County, IL.
<br>
<br>
Given that Cook County is one of the most populous counties in the U.S. (the third most populous, to be exact), there are large inflows and outflows into the county and consequently, a large number of origins and destinations for migration into and out of the county. By analyzing the data and determining the top origins and destinations for migration inflows and outflows, we will determine probabilities of migrating from Cook County to a certain destination and vice versa. Using this information we will create a transition matrix to model the Markov process. We will then create models that predict what the distribution of populations will look like over different periods and what it will look like over the long-term as it converges to the steady state.
<br>
<br>
All the data we are using comes from the US Census Bureau wbsite, specifically the American Community Survey. This data estimates migration patterns between counties, metropolitan areas, and other statistical regions. We selected the data sets for count-to--county migrations available [here](https://www.census.gov/topics/population/migration/guidance/county-to-county-migration-flows.html "County-to-County Migration Flows") for time periods satarting from 2007 through 2016 (the last year available).

<img src="MigrationMap2.jpg" alt="Map" style="width:800px;"/>

In [1]:
import numpy as np
import pandas as pd
from scipy import linalg as LA
import os

### <p style="text-align: center;">Step 1 - Data Acquisition and Setup</p>
---

Set the file path to the working directory and read the file into Python:

In [2]:
file = os.getcwd()+'\\Datasets\\'+'county-to-county-2012-2016-ins-outs-nets-gross.xlsx'
#file

The data is available in an Excel file with multiple worksheets (one for each state and Puerto Rico, etc.). We only need the state of Illinois so we can use the `read_excel` function to import the data and use only the worksheet for Illinois. We can also omit the first four columns of the worksheet which use the FIPS codes (we will not be using these). Here we are using data from the 2012-2016 5-year period.

In [3]:
cols_to_import = list(range(4,16))
IL_to_from = pd.read_excel(file, sheet_name='Illinois', header=1, usecols=cols_to_import, na_values='NA')
IL_to_from.head()

,State Name of Geography A,County Name of Geography A,State/U.S. Island Area/Foreign Region of Geography B,County Name of Geography B,Flow from Geography B to Geography A,Unnamed: 9,Counterflow from Geography A to Geography B1,Unnamed: 11,Net Migration from Geography B to Geography A1,Unnamed: 13,Gross Migration between Geography A and Geography B1,Unnamed: 15
0,NaN,NaN,NaN,NaN,Estimate,MOE,Estimate,MOE,Estimate,MOE,Estimate,MOE
1,Illinois,Adams County,Alabama,Baldwin County,17,23,0,27,17,23,17,23
2,Illinois,Adams County,Alabama,Colbert County,5,11,0,27,5,11,5,11
3,Illinois,Adams County,Alabama,Russell County,50,69,0,27,50,69,50,69
4,Illinois,Adams County,Alaska,Ketchikan Gateway Borough,7,11,0,14,7,11,7,11


"MOE" stands for [Margin of Error](https://www.census.gov/programs-surveys/acs/guidance/training-presentations/acs-moe.html). The numbers are all estimates that are calculated using a special wieghted formula based on the sampling that was conducted. Since we will not be conducting statistical testing for this project, these values are not useful for our purpose and should not have a significant impact on the larger aggregated values we will use, so we will remove those colmns as well.

In [4]:
IL_to_from.drop(['Unnamed: 9', 'Unnamed: 11', 'Unnamed: 13', 'Unnamed: 15'], axis=1, inplace=True)
IL_to_from.head()

,State Name of Geography A,County Name of Geography A,State/U.S. Island Area/Foreign Region of Geography B,County Name of Geography B,Flow from Geography B to Geography A,Counterflow from Geography A to Geography B1,Net Migration from Geography B to Geography A1,Gross Migration between Geography A and Geography B1
0,NaN,NaN,NaN,NaN,Estimate,Estimate,Estimate,Estimate
1,Illinois,Adams County,Alabama,Baldwin County,17,0,17,17
2,Illinois,Adams County,Alabama,Colbert County,5,0,5,5
3,Illinois,Adams County,Alabama,Russell County,50,0,50,50
4,Illinois,Adams County,Alaska,Ketchikan Gateway Borough,7,0,7,7


We can also remove the first row of the frame.

In [5]:
IL_to_from.drop(index=0, inplace=True)
IL_to_from.head()

,State Name of Geography A,County Name of Geography A,State/U.S. Island Area/Foreign Region of Geography B,County Name of Geography B,Flow from Geography B to Geography A,Counterflow from Geography A to Geography B1,Net Migration from Geography B to Geography A1,Gross Migration between Geography A and Geography B1
1,Illinois,Adams County,Alabama,Baldwin County,17,0,17,17
2,Illinois,Adams County,Alabama,Colbert County,5,0,5,5
3,Illinois,Adams County,Alabama,Russell County,50,0,50,50
4,Illinois,Adams County,Alaska,Ketchikan Gateway Borough,7,0,7,7
5,Illinois,Adams County,Arizona,Pima County,0,12,-12,12


---
Now we create a subset of the data that only includes Cook County since that is our focus. We create a deep copy to insure that this new dataframe is independent of the original data frame to avoid any problems.

In [6]:
CookCountyFlows = IL_to_from.loc[(IL_to_from['County Name of Geography A'] == 'Cook County'), :].copy(deep=True)
CookCountyFlows.index = range(1, len(CookCountyFlows.index)+1)  # Index again sarting from 1
CookCountyFlows.head()

,State Name of Geography A,County Name of Geography A,State/U.S. Island Area/Foreign Region of Geography B,County Name of Geography B,Flow from Geography B to Geography A,Counterflow from Geography A to Geography B1,Net Migration from Geography B to Geography A1,Gross Migration between Geography A and Geography B1
1,Illinois,Cook County,Alabama,Autauga County,0,11,-11,11
2,Illinois,Cook County,Alabama,Baldwin County,23,0,23,23
3,Illinois,Cook County,Alabama,Butler County,17,0,17,17
4,Illinois,Cook County,Alabama,Calhoun County,0,6,-6,6
5,Illinois,Cook County,Alabama,Conecuh County,0,10,-10,10


In [7]:
CookCountyFlows.tail()

,State Name of Geography A,County Name of Geography A,State/U.S. Island Area/Foreign Region of Geography B,County Name of Geography B,Flow from Geography B to Geography A,Counterflow from Geography A to Geography B1,Net Migration from Geography B to Geography A1,Gross Migration between Geography A and Geography B1
1331,Illinois,Cook County,Europe,-,7714,NaN,NaN,NaN
1332,Illinois,Cook County,U.S. Island Areas,-,97,NaN,NaN,NaN
1333,Illinois,Cook County,Northern America,-,1221,NaN,NaN,NaN
1334,Illinois,Cook County,Oceania and At Sea,-,232,NaN,NaN,NaN
1335,Illinois,Cook County,South America,-,1971,NaN,NaN,NaN


The last few rows include migration from abroad. Some data, such as outflow migration, is unavailable. We can decide later whether we want to focus on domestic migration only or include international migration as well. If we include international migration, we will have to figure out how to handle the NA values.

Eliminate the migration from abroad, because we are only concentrating on domestic migration. Notice that the rows for immigration from abroad are the only ones with NA values in the columns. We can simply drop any rows with any NA values to eliminate migration from abroad.

In [8]:
CookCountyFlows.dropna(axis=0, inplace=True)
CookCountyFlows_domestic = CookCountyFlows.copy(deep=True)
CookCountyFlows_domestic.tail()

,State Name of Geography A,County Name of Geography A,State/U.S. Island Area/Foreign Region of Geography B,County Name of Geography B,Flow from Geography B to Geography A,Counterflow from Geography A to Geography B1,Net Migration from Geography B to Geography A1,Gross Migration between Geography A and Geography B1
1322,Illinois,Cook County,Puerto Rico,Toa Baja Municipio,19,0,19,19
1323,Illinois,Cook County,Puerto Rico,Trujillo Alto Municipio,0,55,-55,55
1324,Illinois,Cook County,Puerto Rico,Vega Baja Municipio,23,0,23,23
1325,Illinois,Cook County,Puerto Rico,Villalba Municipio,23,16,7,39
1326,Illinois,Cook County,Puerto Rico,Yauco Municipio,29,0,29,29


We also decided to exclude migration from Puerto Rico for this study and focus only on migration to and from locations in the 50 states.

In [9]:
removePR = CookCountyFlows_domestic.loc[(CookCountyFlows['State/U.S. Island Area/Foreign Region of Geography B']==
                                        'Puerto Rico'), :].index
CookCountyFlows_domestic.drop(labels=list(removePR), axis=0, inplace=True)
CookCountyFlows_domestic.tail()

,State Name of Geography A,County Name of Geography A,State/U.S. Island Area/Foreign Region of Geography B,County Name of Geography B,Flow from Geography B to Geography A,Counterflow from Geography A to Geography B1,Net Migration from Geography B to Geography A1,Gross Migration between Geography A and Geography B1
1290,Illinois,Cook County,Wyoming,Campbell County,9,66,-57,75
1291,Illinois,Cook County,Wyoming,Laramie County,0,2,-2,2
1292,Illinois,Cook County,Wyoming,Lincoln County,0,41,-41,41
1293,Illinois,Cook County,Wyoming,Park County,0,20,-20,20
1294,Illinois,Cook County,Wyoming,Teton County,0,6,-6,6


Rename the columns to make them more meaningful for this study.

In [10]:
rename_cols = dict(zip(list(CookCountyFlows_domestic.columns), 
                       ["State", "County", "Source or Destination", "Source or Dest County", "Inbound Migration", 
                        "Outbound Migration","Net Migration", "Total Migration"]))
rename_cols

{'State Name of Geography A': 'State',
 'County Name of Geography A': 'County',
 'State/U.S. Island Area/Foreign Region of Geography B': 'Source or Destination',
 'County Name of Geography B': 'Source or Dest County',
 'Flow from Geography B to Geography A': 'Inbound Migration',
 'Counterflow from Geography A to Geography B1': 'Outbound Migration',
 'Net Migration from Geography B to Geography A1': 'Net Migration',
 'Gross Migration between Geography A and Geography B1': 'Total Migration'}

In [11]:
CookCountyFlows_domestic.rename(index=str, columns=rename_cols, inplace=True)
CookCountyFlows_domestic.head()

,State,County,Source or Destination,Source or Dest County,Inbound Migration,Outbound Migration,Net Migration,Total Migration
1,Illinois,Cook County,Alabama,Autauga County,0,11,-11,11
2,Illinois,Cook County,Alabama,Baldwin County,23,0,23,23
3,Illinois,Cook County,Alabama,Butler County,17,0,17,17
4,Illinois,Cook County,Alabama,Calhoun County,0,6,-6,6
5,Illinois,Cook County,Alabama,Conecuh County,0,10,-10,10


Import population dataset for Cook County and other counties.

In [12]:
file = 'Datasets\\county-to-county-2012-2016-current-residence-sort.xlsx'
cols_to_import = [4, 5, 6, 20, 21, 22]
countiespopulation = pd.read_excel(file, sheet_name='Illinois', header=1, usecols="E:G,U:W", na_values='NA', 
                                   skiprows=[2,3])
countiespopulation.head()

,State of Current Residence,County of Current Residence,County of Current Residence.1,State/U.S. Island Area/Foreign Region of Residence 1 Year Ago,County of Residence 1 Year Ago,County of Residence 1 Year Ago.1
0,Illinois,Adams County,65944.0,Alabama,Baldwin County,194243.0
1,Illinois,Adams County,65944.0,Alabama,Colbert County,53937.0
2,Illinois,Adams County,65944.0,Alabama,Russell County,57100.0
3,Illinois,Adams County,65944.0,Alaska,Ketchikan Gateway Borough,14067.0
4,Illinois,Adams County,65944.0,Arizona,Pinal County,378676.0


We will assume that these populations will serve as our starting populations as well as the basis for migration probabilities.

In [46]:
StartingPops = countiespopulation.loc[(countiespopulation['County of Current Residence'] == 'Cook County'), 
                                      :].copy(deep=True)
StartingPops.head()

,State of Current Residence,County of Current Residence,County of Current Residence.1,State/U.S. Island Area/Foreign Region of Residence 1 Year Ago,County of Residence 1 Year Ago,County of Residence 1 Year Ago.1
984,Illinois,Cook County,5162969.0,Alabama,Baldwin County,194243.0
985,Illinois,Cook County,5162969.0,Alabama,Butler County,20697.0
986,Illinois,Cook County,5162969.0,Alabama,Jefferson County,650080.0
987,Illinois,Cook County,5162969.0,Alabama,Lee County,146817.0
988,Illinois,Cook County,5162969.0,Alabama,Madison County,343931.0


Here, we rename the columns again. This is useful not only because it gives more meaningful names but more importantly, we will use them when merging the dataframes for migration flows and starting populations.

In [47]:
popname_cols = dict(zip(list(StartingPops.columns), ["State", "County", "Population of Current County", 
                                                     "Source or Destination", "Source or Dest County", 
                                                     "Population of Dest County"]))
StartingPops.rename(index=str, columns=popname_cols, inplace=True)
StartingPops.head()

,State,County,Population of Current County,Source or Destination,Source or Dest County,Population of Dest County
984,Illinois,Cook County,5162969.0,Alabama,Baldwin County,194243.0
985,Illinois,Cook County,5162969.0,Alabama,Butler County,20697.0
986,Illinois,Cook County,5162969.0,Alabama,Jefferson County,650080.0
987,Illinois,Cook County,5162969.0,Alabama,Lee County,146817.0
988,Illinois,Cook County,5162969.0,Alabama,Madison County,343931.0


In [48]:
StartingPops.tail()

,State,County,Population of Current County,Source or Destination,Source or Dest County,Population of Dest County
1803,Illinois,Cook County,5162969.0,Europe,-,NaN
1804,Illinois,Cook County,5162969.0,U.S. Island Areas,-,NaN
1805,Illinois,Cook County,5162969.0,Northern America,-,NaN
1806,Illinois,Cook County,5162969.0,Oceania and At Sea,-,NaN
1807,Illinois,Cook County,5162969.0,South America,-,NaN


Merge the two dataframes into a single dataframe with flows and starting populations. Sort the new dataframe by total migration between counties.

In [49]:
Flow_vs_Pops = pd.merge(CookCountyFlows_domestic, StartingPops, how='inner', 
                        on=['County','State','Source or Destination','Source or Dest County'])
Flow_vs_Pops.sort_values('Total Migration', ascending=False, inplace=True)

In [50]:
Flow_vs_Pops.head()

,State,County,Source or Destination,Source or Dest County,Inbound Migration,Outbound Migration,Net Migration,Total Migration,Population of Current County,Population of Dest County
162,Illinois,Cook County,Illinois,DuPage County,15212,19163,-3951,34375,5162969.0,923374.0
180,Illinois,Cook County,Illinois,Lake County,9483,11276,-1793,20759,5162969.0,696463.0
219,Illinois,Cook County,Illinois,Will County,6321,10362,-4041,16683,5162969.0,682331.0
238,Illinois,Cook County,Indiana,Lake County,3440,9896,-6456,13336,5162969.0,483179.0
176,Illinois,Cook County,Illinois,Kane County,5488,7010,-1522,12498,5162969.0,522146.0


It is reasonable to expect that a relatively limited number of locations will account for the largest population flows into and out of Cook County. We can use the 80/20 rule here to work out the top destinations that will account for about 80% of the migration flow and just lump the remaining counties into a separate group.

First, let's reorder the columns and drop the "Net Migration" columns since this will be calculated by our model later.

In [51]:
Flow_vs_Pops.drop(columns='Net Migration', inplace=True)
Flow_vs_Pops.head()

,State,County,Source or Destination,Source or Dest County,Inbound Migration,Outbound Migration,Total Migration,Population of Current County,Population of Dest County
162,Illinois,Cook County,Illinois,DuPage County,15212,19163,34375,5162969.0,923374.0
180,Illinois,Cook County,Illinois,Lake County,9483,11276,20759,5162969.0,696463.0
219,Illinois,Cook County,Illinois,Will County,6321,10362,16683,5162969.0,682331.0
238,Illinois,Cook County,Indiana,Lake County,3440,9896,13336,5162969.0,483179.0
176,Illinois,Cook County,Illinois,Kane County,5488,7010,12498,5162969.0,522146.0


In [55]:
#new_list = list(Flow_vs_Pops.columns)
#new_list.pop(new_list.index('Total Migration'))
#new_list.append('Total Migration')
#Flow_vs_Pops = Flow_vs_Pops[new_list]

In [56]:
Flow_vs_Pops.head()

,State,County,Source or Destination,Source or Dest County,Inbound Migration,Outbound Migration,Population of Current County,Population of Dest County,Total Migration
162,Illinois,Cook County,Illinois,DuPage County,15212,19163,5162969.0,923374.0,34375
180,Illinois,Cook County,Illinois,Lake County,9483,11276,5162969.0,696463.0,20759
219,Illinois,Cook County,Illinois,Will County,6321,10362,5162969.0,682331.0,16683
238,Illinois,Cook County,Indiana,Lake County,3440,9896,5162969.0,483179.0,13336
176,Illinois,Cook County,Illinois,Kane County,5488,7010,5162969.0,522146.0,12498


Add a new columns to determine the cutoff for the top migration locations:

In [58]:
Flow_vs_Pops['Cumulative Percentage of Total Migration'] = \
    Flow_vs_Pops['Total Migration'].cumsum()/sum(Flow_vs_Pops['Total Migration'])

In [61]:
Flow_vs_Pops.head()

,State,County,Source or Destination,Source or Dest County,Inbound Migration,Outbound Migration,Population of Current County,Population of Dest County,Total Migration,Cumulative Percentage of Total Migration
162,Illinois,Cook County,Illinois,DuPage County,15212,19163,5162969.0,923374.0,34375,0.101576
180,Illinois,Cook County,Illinois,Lake County,9483,11276,5162969.0,696463.0,20759,0.162917
219,Illinois,Cook County,Illinois,Will County,6321,10362,5162969.0,682331.0,16683,0.212215
238,Illinois,Cook County,Indiana,Lake County,3440,9896,5162969.0,483179.0,13336,0.251622
176,Illinois,Cook County,Illinois,Kane County,5488,7010,5162969.0,522146.0,12498,0.288552


We can see that five counties alone account for over 25% of the migration activity to and from Cook County.

Split the dataframe into two separate frames, one representing the top 80% of migration regions, the other representing the remaing 20%.

In [62]:
TopMigrationRegions = Flow_vs_Pops.loc[(Flow_vs_Pops['Cumulative Percentage of Total Migration']) <= 0.8]
OtherRegions = Flow_vs_Pops.loc[(Flow_vs_Pops['Cumulative Percentage of Total Migration']) > 0.8]

In [63]:
TopMigrationRegions.tail()

,State,County,Source or Destination,Source or Dest County,Inbound Migration,Outbound Migration,Population of Current County,Population of Dest County,Total Migration,Cumulative Percentage of Total Migration
519,Illinois,Cook County,New York,Westchester County,278,243,5162969.0,957081.0,521,0.792827
714,Illinois,Cook County,Virginia,Henrico County,93,427,5162969.0,313552.0,520,0.794363
341,Illinois,Cook County,Massachusetts,Norfolk County,339,178,5162969.0,679828.0,517,0.795891
281,Illinois,Cook County,Iowa,Scott County,191,324,5162969.0,170011.0,515,0.797413
709,Illinois,Cook County,Virginia,Arlington County,249,265,5162969.0,220506.0,514,0.798931


In [64]:
OtherRegions.head()

,State,County,Source or Destination,Source or Dest County,Inbound Migration,Outbound Migration,Population of Current County,Population of Dest County,Total Migration,Cumulative Percentage of Total Migration
55,Illinois,Cook County,California,San Mateo County,176,335,5162969.0,741798.0,511,0.800441
49,Illinois,Cook County,California,Riverside County,141,365,5162969.0,2265999.0,506,0.801937
486,Illinois,Cook County,New Mexico,Bernalillo County,373,129,5162969.0,662286.0,502,0.80342
111,Illinois,Cook County,Florida,Pinellas County,171,314,5162969.0,916830.0,485,0.804853
439,Illinois,Cook County,Missouri,Jackson County,224,260,5162969.0,676661.0,484,0.806283


In [69]:
len(TopMigrationRegions)

127

In [67]:
len(OtherRegions)

665

Let's sum up the lower 20% and put everything together in a new dataframe called "TotalMigrations".

In [99]:
summary_vals = ['Illinois', 'Cook County', 'All Other States', 'Any', sum(OtherRegions['Inbound Migration']),
          sum(OtherRegions['Outbound Migration']), np.mean(OtherRegions['Population of Current County']),
          sum(OtherRegions['Population of Dest County']), sum(OtherRegions['Total Migration']),
          (1 - TopMigrationRegions.iloc[len(TopMigrationRegions)-1]['Cumulative Percentage of Total Migration'])]
others_dict = dict(zip(list(OtherRegions.columns), summary_vals))
OtherLocations = pd.Series(others_dict)
OtherLocations

State                                               Illinois
County                                           Cook County
Source or Destination                       All Other States
Source or Dest County                                    Any
Inbound Migration                                      31932
Outbound Migration                                     36113
Population of Current County                     5.16297e+06
Population of Dest County                        1.20337e+08
Total Migration                                        68045
Cumulative Percentage of Total Migration            0.201069
dtype: object

In [100]:
TotalMigrations = TopMigrationRegions.append(OtherLocations, ignore_index=True)

In [101]:
TotalMigrations.tail()

,State,County,Source or Destination,Source or Dest County,Inbound Migration,Outbound Migration,Population of Current County,Population of Dest County,Total Migration,Cumulative Percentage of Total Migration
123,Illinois,Cook County,Virginia,Henrico County,93,427,5162969.0,313552.0,520,0.794363
124,Illinois,Cook County,Massachusetts,Norfolk County,339,178,5162969.0,679828.0,517,0.795891
125,Illinois,Cook County,Iowa,Scott County,191,324,5162969.0,170011.0,515,0.797413
126,Illinois,Cook County,Virginia,Arlington County,249,265,5162969.0,220506.0,514,0.798931
127,Illinois,Cook County,All Other States,Any,31932,36113,5162969.0,120337165.0,68045,0.201069


Now to set up the probabilities. We assume that the current estimated populations in our original data are the starting populations and the migration inflows and outflows between Cook County and the other location are steady. Therefore, dividing the inflow by source population and outflow by Cook County population gives us the probability of migration from the source to Cook County and vice versa.

In [102]:
TotalMigrations.drop(columns=['Total Migration', 'Cumulative Percentage of Total Migration'], inplace=True)

In [104]:
TotalMigrations['Prob. Other to Cook Cty'] = \
    TotalMigrations['Inbound Migration']/TotalMigrations['Population of Dest County']
TotalMigrations['Prob. Cook Cty to Other'] = \
    TotalMigrations['Outbound Migration']/TotalMigrations['Population of Current County']

In [105]:
TotalMigrations.head()

,State,County,Source or Destination,Source or Dest County,Inbound Migration,Outbound Migration,Population of Current County,Population of Dest County,Prob. Other to Cook Cty,Prob. Cook Cty to Other
0,Illinois,Cook County,Illinois,DuPage County,15212,19163,5162969.0,923374.0,0.0164744,0.00371162
1,Illinois,Cook County,Illinois,Lake County,9483,11276,5162969.0,696463.0,0.0136159,0.00218401
2,Illinois,Cook County,Illinois,Will County,6321,10362,5162969.0,682331.0,0.00926383,0.00200698
3,Illinois,Cook County,Indiana,Lake County,3440,9896,5162969.0,483179.0,0.00711951,0.00191673
4,Illinois,Cook County,Illinois,Kane County,5488,7010,5162969.0,522146.0,0.0105105,0.00135775


---
As a final data preparation step, let's create a new dataframe for extracting our Markov chains with only the relevant data.

In [121]:
TransitionData = TotalMigrations.copy(deep=True)
TransitionData.drop(columns=['State', 'County', 'Inbound Migration', 'Outbound Migration'], inplace=True)

In [122]:
list(TransitionData.columns)

['Source or Destination',
 'Source or Dest County',
 'Population of Current County',
 'Population of Dest County',
 'Prob. Other to Cook Cty',
 'Prob. Cook Cty to Other']

In [123]:
new_cols = list(TransitionData.columns)
new_cols.pop(new_cols.index('Source or Dest County'))
new_cols.insert(0, 'Source or Dest County')
TransitionData = TransitionData[new_cols]

In [124]:
TransitionData.head()

,Source or Dest County,Source or Destination,Population of Current County,Population of Dest County,Prob. Other to Cook Cty,Prob. Cook Cty to Other
0,DuPage County,Illinois,5162969.0,923374.0,0.0164744,0.00371162
1,Lake County,Illinois,5162969.0,696463.0,0.0136159,0.00218401
2,Will County,Illinois,5162969.0,682331.0,0.00926383,0.00200698
3,Lake County,Indiana,5162969.0,483179.0,0.00711951,0.00191673
4,Kane County,Illinois,5162969.0,522146.0,0.0105105,0.00135775


Write the transition data out to a file for future use if needed.

In [125]:
TransitionData.to_csv(os.getcwd()+'\\Datasets\\TransitionData.csv')

---
### Markov Chains
---

We can now implement the Markov chain calculations. In this model, we will use a Markov chain to simulate the migration flows between Cook County and other locations. A simple chain would have the form of the image below:

<img src="MarkovChain.gif" alt="Markov Chain" style="width:200px;"/>

Image source: [Setosa.io](http://setosa.io/ev/markov-chains)

Using Cook County as  point "A" for example, and any other location as "B", the result is a $2 \times 2$ Markov transition matrix for Cook County and *each* of the other locations in our TransitionData dataframe with each column representing the probability of moving to the other location vs not moving to the other location:


\begin{array}{|c|c|c|}
\hline
  & Cook\;County & Other\;Location \\ 
  \hline
  Cook\;County & 1-P(Other | Cook\;Co) & P(Cook\;Co | Other) \\
  \hline
  Other\;Location & P(Other | Cook\;Co) & 1-P(Cook\;Co | Other) \\
  \hline
 \end{array}


Multiplying the Markov matrix $A$ by the a vector representing the initial populations $P_o$ will give us the population for one period (in this case, one year). For $n$ years, we need to multiply by the Markov matrix $n$ times, i.e. $A^n$.

After setting up the Markov matrix, we can solve for the eigenvalues and eigenvectors and decompose our matrix to the spectral $\mathbf{A}=\mathbf{\mathit{S}\mathit{\Lambda}\mathit{S^{-1}}}$. It is now easy to calculate multiple years because $\Lambda^n$ is simply the eigenvalues raised to the power of $n$.

We have 128 different locations and our output will include prdictions for 1-year, 5-year and 10-year periods. This means we will be solving a total of $128\times3$ equations.

---
### 1)
Define a function to build the transition matrix:

In [159]:
def MarkovMatrix(row, df=TransitionData):
#'''Transforms any row of the data frame into Markov transition matrix'''
    Po = np.array([[df.at[row, 'Population of Current County']],
                  [df.at[row, 'Population of Dest County']]])  # Create vector of initial populations

    A = np.array([[1 - df.at[row, 'Prob. Cook Cty to Other'],
                   df.at[row, 'Prob. Other to Cook Cty']],
                  [df.at[row, 'Prob. Cook Cty to Other'],
                   1 - df.at[row, 'Prob. Other to Cook Cty']]])  # Markov matrix

    return(A, Po)

### 2)
Define a function to decompose the Markov matrix and apply the transformation:

In [160]:
def Transformation(A, n):
    eigVal_A, eigVec_A = LA.eig(A)
    LambdaMatrix = np.diag(np.real(eigVal_A))
    SMatrix = eigVec_A
    SMatrixInv = LA.inv(SMatrix)
    A_transformed = SMatrix @ np.power(LambdaMatrix, n) @ SMatrixInv

    return(A_transformed)

### 3)
Create a new data frame that will hold our calculated results:

In [167]:
OutputDict = {('Net Population Change', '1 Year'): [0]*len(TransitionData),
              ('Net Population Change', '5 Years'): [0]*len(TransitionData),
              ('Net Population Change', '10 Years'): [0]*len(TransitionData)}
PopulationPredictions = pd.DataFrame(OutputDict)
PopulationPredictions.insert(loc=0, column=('Migrating To or From', 'County'),
                                            value=[0]*len(TransitionData))
PopulationPredictions.insert(loc=1, column=('Migrating To or From', 'State'),
                                            value=[0]*len(TransitionData))

### 4)
Run the predictions and add them to our new data frame `PopulationPredictions`:

In [168]:
for i in range(len(TransitionData)):
    values = []
    values.append(TransitionData.loc[i, 'Source or Dest County'])
    values.append(TransitionData.loc[i, 'Source or Destination'])

    A, Po = MarkovMatrix(i)

    change_1yr = Transformation(A, 1) @ Po
    change_5yr = Transformation(A, 5) @ Po
    change_10yr = Transformation(A, 10) @ Po
    values.extend([round(change_1yr.item(0) - Po.item(0), 0),
                   round(change_5yr.item(0) - Po.item(0), 0),
                   round(change_10yr.item(0) - Po.item(0), 0)])
    
    PopulationPredictions.loc[i] = values

In [174]:
PopulationPredictions

Migrating To or From                       Net Population Change  \
                   County                 State                1 Year   
0           DuPage County              Illinois               -3951.0   
1             Lake County              Illinois               -1793.0   
2             Will County              Illinois               -4041.0   
3             Lake County               Indiana               -6456.0   
4             Kane County              Illinois               -1522.0   
5        Champaign County              Illinois               -2350.0   
6      Los Angeles County            California               -1022.0   
7          McHenry County              Illinois               -1049.0   
8         Maricopa County               Arizona               -1572.0   
9        Milwaukee County             Wisconsin                -758.0   
10        New York County              New York                 -46.0   
11          Harris County                 Texas               -1817.0   
12          DeKalb County              Illinois               -1520.0   
13        Hennepin County             Minnesota                -489.0   
14          McLean County              Illinois               -1093.0   
15            Dane County             Wisconsin                -657.0   
16            King County            Washington               -1002.0   
17       Winnebago County              Illinois                -663.0   
18          Dallas County                 Texas               -1147.0   
19       San Diego County            California                -146.0   
20   San Francisco County            California                -827.0   
21           Clark County                Nevada                -650.0   
22          Fulton County               Georgia                 277.0   
23          Orange County            California               -1030.0   
24          Marion County               Indiana                -673.0   
25          Orange County               Florida                -365.0   
26   District of Columbia  District of Columbia                -323.0   
27      St. Joseph County               Indiana                 246.0   
28           Wayne County              Michigan                 358.0   
29         Oakland County              Michigan                 417.0   
..                    ...                   ...                   ...   
98            Rock County             Wisconsin                -383.0   
99            Linn County                  Iowa                  78.0   
100        Douglas County              Nebraska                 -30.0   
101        Orleans Parish             Louisiana                 -59.0   
102       Honolulu County                Hawaii                  -8.0   
103         Racine County             Wisconsin                -321.0   
104      Jefferson County              Kentucky                -325.0   
105         Denton County                 Texas                -298.0   
106        El Paso County              Colorado                 -91.0   
107          Boone County              Missouri                -245.0   
108           Wake County        North Carolina                -310.0   
109      St. Clair County              Illinois                 -29.0   
110        LaPorte County               Indiana                -389.0   
111          Boone County              Illinois                -222.0   
112        Fayette County              Illinois                -376.0   
113          Bexar County                 Texas                 -32.0   
114         Monroe County              New York                -103.0   
115       Gwinnett County               Georgia                -109.0   
116         Morris County            New Jersey                 271.0   
117         Nassau County              New York                -351.0   
118        Berrien County              Michigan                -127.0   
119        Baltimore city              Maryland                

This table summarizes the net change from migrations to and from that could happen over **one**, **five** and **ten** years assuming the rate of inflows/outflows remains constant over these periods.

---
We can apply some additional data frame maniputlations to get more information from the data.

Let's get the total change in Cook County population expected over the next one, five and ten years. This is the *sum* of all the net population changes.

In [230]:
yr1 = PopulationPredictions.loc[:, ('Net Population Change', '1 Year')].sum()
yr5 = PopulationPredictions.loc[:, ('Net Population Change', '5 Years')].sum()
yr10 = PopulationPredictions.loc[:, ('Net Population Change', '10 Years')].sum()

print("Overall change in Cook County Population:\nIn one year, the population changes by", yr1,
     "and the new population is", TransitionData.at[0,'Population of Current County'] + yr1,
     "\nIn five years, the population changes by", yr5,
     "and the new population is", TransitionData.at[0,'Population of Current County'] + yr5,
     "\nIn ten years, population changes by", yr10,
     "and the new population is", TransitionData.at[0,'Population of Current County'] + yr10)

Overall change in Cook County Population:
In one year, the population changes by -63479.0 and the new population is 5099490.0 
In five years, the population changes by -313657.0 and the new population is 4849312.0 
In ten years, population changes by -618307.0 and the new population is 4544662.0


We can also look at the changes by state instead of at the county level:

In [231]:
MigrationbyState = PopulationPredictions.pivot_table(values=[('Net Population Change', '1 Year'),
                                                            ('Net Population Change', '5 Years'),
                                                            ('Net Population Change', '10 Years')],
                                                     index=[('Migrating To or From','State')], aggfunc='sum')

In [240]:
MigrationbyState.head()

Net Population Change                  
                                             1 Year 10 Years  5 Years
(Migrating To or From, State)                                        
Alaska                                        261.0   2592.0   1301.0
All Other States                            -4181.0 -40470.0 -20604.0
Arizona                                     -1919.0 -19115.0  -9579.0
California                                  -4420.0 -44066.0 -22069.0
Colorado                                    -1987.0 -19827.0  -9926.0

Let's say we are only interested in the top 5 states that have the biggest effect on the population of Cook County over 10 years.

In [238]:
ByState10yr = PopulationPredictions.pivot_table(values=[('Net Population Change', '10 Years')],
                                                     index=[('Migrating To or From','State')], aggfunc='sum')
ByState10yr.sort_values(('Net Population Change', '10 Years'))[:5]

,Net Population Change
,10 Years
"(Migrating To or From, State)",
Illinois,-234822.0
Indiana,-93191.0
California,-44066.0
Texas,-43062.0
All Other States,-40470.0
